# 第04章：Structured Output（结构化输出）

## 本章学习目标

1. 理解为什么需要结构化输出
2. 掌握 with_structured_output() 方法的使用
3. 学会使用 Pydantic 模型定义输出schema
4. 了解 TypedDict 作为简化替代方案
5. 掌握 include_raw 参数获取原始响应
6. 构建复杂嵌套结构的输出
7. 理解 JSON Schema 方式
8. 了解传统 Output Parser（历史对比）

## 核心概念

- **结构化输出**：让AI按预定格式返回数据，而非自由文本
- **with_structured_output()**：LangChain v1的标准结构化输出方法
- **Pydantic模型**：定义类型安全的输出schema
- **TypedDict**：轻量级的类型注解替代方案
- **include_raw**：同时获取结构化数据和原始响应
- **嵌套结构**：处理复杂的多层数据结构

## 为什么重要？

从第03章的LCEL，我们学会了组合组件。现在我们要让AI的输出变得**可靠、可编程、类型安全**！

**对比传统方式**：
- 传统：AI返回字符串 → 手动解析 → 容易出错
- 结构化：AI返回对象 → 直接使用 → 类型安全


In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 导入结构化输出相关模块
from typing import List, Optional, Dict, Any, Union
from pydantic import BaseModel, Field, validator
from typing_extensions import TypedDict

# 初始化模型
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,  # 降低温度获得更稳定的结构化输出
    api_key=config.OPENAI_API_KEY,
    base_url=config.OPENAI_BASE_URL,
)

print("环境配置完成")
print("结构化输出学习开始！")
print(f"模型: {model.model_name}")
print(f"支持结构化输出: {hasattr(model, 'with_structured_output')}")


环境配置完成
结构化输出学习开始！
模型: gpt-4o-mini
支持结构化输出: True


## 1. 为什么需要结构化输出？

### 传统方式的问题

在没有结构化输出之前，我们需要手动解析AI的文本回复：

**问题场景**：
- AI返回自由文本，格式不统一
- 需要写复杂的字符串解析逻辑
- 容易因格式变化而出错
- 无法保证数据类型安全

**示例对比**：让AI分析一个人的信息


In [3]:
# 传统方式：不可靠的文本解析
print("【传统方式的问题演示】")

prompt = ChatPromptTemplate.from_template("""
分析以下人员信息，返回姓名、年龄、职业和技能列表：

信息：{person_info}

请按照以下格式返回：
姓名：...
年龄：...
职业：...
技能：...
""")

chain = prompt | model

# 测试几次，看看输出格式的一致性
test_info = "李明在北京一家互联网公司工作，今年刚满28岁，主要负责前端开发和用户体验设计，平时喜欢研究新技术，会使用React、Vue、TypeScript等框架，还懂一些Python和机器学习"

print("进行3次相同的查询，观察输出格式：\n")

for i in range(3):
    result = chain.invoke({"person_info": test_info})
    print(f"第{i+1}次输出：")
    print(result.content)
    print("-" * 50)

print("\n❌ 问题总结：")
print("1. 每次输出格式可能略有不同")
print("2. 需要写解析代码来提取具体字段") 
print("3. 如果AI改变格式，解析就会失败")
print("4. 无法保证数据类型（比如年龄是数字）")

【传统方式的问题演示】
进行3次相同的查询，观察输出格式：

第1次输出：
姓名：李明  
年龄：28  
职业：前端开发工程师  
技能：React, Vue, TypeScript, Python, 机器学习  
--------------------------------------------------
第2次输出：
姓名：李明  
年龄：28  
职业：前端开发工程师  
技能：React, Vue, TypeScript, Python, 机器学习  
--------------------------------------------------
第3次输出：
姓名：李明  
年龄：28  
职业：前端开发工程师  
技能：React, Vue, TypeScript, Python, 机器学习  
--------------------------------------------------

❌ 问题总结：
1. 每次输出格式可能略有不同
2. 需要写解析代码来提取具体字段
3. 如果AI改变格式，解析就会失败
4. 无法保证数据类型（比如年龄是数字）


## 2. with_structured_output() 解决方案

### 核心概念

`with_structured_output()` 是 LangChain v1 中的**标准方法**，让AI按照预定义的结构返回数据。

**优势**：
- **类型安全**：确保输出符合预期类型
- **格式统一**：每次输出格式完全一致
- **自动验证**：自动检查数据有效性
- **易于使用**：直接得到Python对象

### 基本工作流程

```python
# 1. 定义输出结构（Pydantic模型）
class MyOutputSchema(BaseModel):
    field1: str
    field2: int

# 2. 绑定到模型
structured_model = model.with_structured_output(MyOutputSchema)

# 3. 调用并获得结构化对象
result = structured_model.invoke("your prompt")
# result 是 MyOutputSchema 对象，不是字符串！
```


In [4]:
# 第一个结构化输出示例
print("【结构化输出解决方案】")

# 1. 定义 Pydantic 模型
class PersonInfo(BaseModel):
    """人员信息模型"""
    name: str = Field(description="人员姓名")
    age: int = Field(description="年龄", ge=0, le=150)  # ge=greater equal, le=less equal
    occupation: str = Field(description="职业")
    skills: List[str] = Field(description="技能列表")

# 2. 创建结构化输出模型
structured_model = model.with_structured_output(PersonInfo)

print("模型定义完成！")
print(f"PersonInfo模型字段: {list(PersonInfo.model_fields.keys())}")

# 3. 测试结构化输出
print("\n进行3次相同的查询，观察输出一致性：\n")

test_info = "张三是一名35岁的软件工程师，擅长Python、JavaScript和数据分析"

for i in range(3):
    result = structured_model.invoke(f"分析以下人员信息：{test_info}")
    print(f"第{i+1}次输出：")
    print(f"  类型: {type(result)}")
    print(f"  姓名: {result.name}")
    print(f"  年龄: {result.age} (类型: {type(result.age)})")
    print(f"  职业: {result.occupation}")
    print(f"  技能: {result.skills}")
    print(f"  原始对象: {result}")
    print("-" * 50)

print("\n✅ 优势总结：")
print("1. 输出格式完全一致")
print("2. 直接得到Python对象，无需解析")
print("3. 自动类型转换（age是int类型）") 
print("4. 字段验证（age必须在0-150之间）")
print("5. IDE支持自动补全和类型检查")


【结构化输出解决方案】
模型定义完成！
PersonInfo模型字段: ['name', 'age', 'occupation', 'skills']

进行3次相同的查询，观察输出一致性：

第1次输出：
  类型: <class '__main__.PersonInfo'>
  姓名: 张三
  年龄: 35 (类型: <class 'int'>)
  职业: 软件工程师
  技能: ['Python', 'JavaScript', '数据分析']
  原始对象: name='张三' age=35 occupation='软件工程师' skills=['Python', 'JavaScript', '数据分析']
--------------------------------------------------
第2次输出：
  类型: <class '__main__.PersonInfo'>
  姓名: 张三
  年龄: 35 (类型: <class 'int'>)
  职业: 软件工程师
  技能: ['Python', 'JavaScript', '数据分析']
  原始对象: name='张三' age=35 occupation='软件工程师' skills=['Python', 'JavaScript', '数据分析']
--------------------------------------------------
第3次输出：
  类型: <class '__main__.PersonInfo'>
  姓名: 张三
  年龄: 35 (类型: <class 'int'>)
  职业: 软件工程师
  技能: ['Python', 'JavaScript', '数据分析']
  原始对象: name='张三' age=35 occupation='软件工程师' skills=['Python', 'JavaScript', '数据分析']
--------------------------------------------------

✅ 优势总结：
1. 输出格式完全一致
2. 直接得到Python对象，无需解析
3. 自动类型转换（age是int类型）
4. 字段验证（age必须在0-150之间）
5. IDE支持自动补全和类型检

## 3. Pydantic 模型深入学习

### Pydantic 基础知识

**Pydantic** 是Python的数据验证库，提供：
- **类型注解**：明确字段类型
- **数据验证**：自动检查数据有效性  
- **序列化/反序列化**：JSON ↔ Python对象
- **文档生成**：自动生成schema文档

### Field 的强大功能

`Field()` 可以添加丰富的验证规则和描述：

| 参数 | 作用 | 示例 |
|------|------|------|
| `description` | 字段描述 | `Field(description="用户姓名")` |
| `ge/le` | 数值范围 | `Field(ge=0, le=100)` # 0≤值≤100 |
| `min_length/max_length` | 字符串长度 | `Field(min_length=2, max_length=50)` |
| `pattern` | 正则表达式 | `Field(pattern=r"^\d{11}$")` # 11位数字 |
| `default` | 默认值 | `Field(default="未知")` |
| `examples` | 示例值 | `Field(examples=["张三", "李四"])` |

In [8]:
# Pydantic 高级功能示例
print("【Pydantic 高级功能演示】")

# 定义一个复杂的产品评论分析模型
class ProductReview(BaseModel):
    """产品评论分析结果"""
    
    # 基本信息
    product_name: str = Field(description="产品名称")
    reviewer_name: Optional[str] = Field(default=None, description="评论者姓名（如果提及）")
    
    # 评分相关
    rating: int = Field(description="评分", ge=1, le=5)  # 1-5星评分
    
    # 情感分析
    sentiment: str = Field(description="整体情感倾向", pattern="^(positive|negative|neutral)$")
    confidence: float = Field(description="情感分析置信度", ge=0.0, le=1.0)
    
    # 详细分析
    pros: List[str] = Field(description="产品优点列表", max_length=10)
    cons: List[str] = Field(description="产品缺点列表", max_length=10)
    
    # 关键词
    keywords: List[str] = Field(description="关键词标签", min_length=1, max_length=20)
    
    # 推荐度
    would_recommend: bool = Field(description="是否推荐该产品")
    
    # 可选的详细信息
    price_mention: Optional[str] = Field(default=None, description="如果提到价格相关内容")
    usage_duration: Optional[str] = Field(default=None, description="使用时长（如果提及）")

# 创建结构化模型
review_analyzer = model.with_structured_output(ProductReview)

# 测试评论分析
test_review = """
我买了这款iPhone 15 Pro已经用了3个月了。整体来说非常满意！
相机质量真的很棒，拍照效果比我之前的手机好太多了。
处理器也很快，玩游戏完全不卡。电池续航也不错，一天用下来没问题。

不过价格确实有点贵，差不多9000块钱。还有就是充电器需要单独买，这点不太喜欢。
总的来说还是推荐购买的，特别是对拍照有要求的用户。

- 小王
"""

print("测试评论内容:")
print(test_review[:100] + "...")
print("\n" + "=" * 50)

# 分析评论
result = review_analyzer.invoke(f"请详细分析这个产品评论：\n{test_review}")

print("\n【分析结果】")
print(f"产品名称: {result.product_name}")
print(f"评论者: {result.reviewer_name}")
print(f"评分: {result.rating}星")
print(f"情感倾向: {result.sentiment} (置信度: {result.confidence:.2f})")
print(f"优点: {result.pros}")
print(f"缺点: {result.cons}")
print(f"关键词: {result.keywords}")
print(f"推荐购买: {'是' if result.would_recommend else '否'}")
print(f"价格相关: {result.price_mention}")
print(f"使用时长: {result.usage_duration}")

print(f"\n原始对象类型: {type(result)}")
print(f"支持JSON序列化: {result.model_dump_json()[:100]}...")


【Pydantic 高级功能演示】
测试评论内容:

我买了这款iPhone 15 Pro已经用了3个月了。整体来说非常满意！
相机质量真的很棒，拍照效果比我之前的手机好太多了。
处理器也很快，玩游戏完全不卡。电池续航也不错，一天用下来没问题。

不过...


【分析结果】
产品名称: iPhone 15 Pro
评论者: 小王
评分: 4星
情感倾向: positive (置信度: 0.90)
优点: ['相机质量很好，拍照效果出色', '处理器速度快，游戏流畅', '电池续航能力强，一天使用无问题']
缺点: ['价格较贵，接近9000元', '充电器需单独购买']
关键词: ['iPhone 15 Pro', '相机质量', '处理器速度', '电池续航', '价格']
推荐购买: 是
价格相关: 9000块钱
使用时长: 3个月

原始对象类型: <class '__main__.ProductReview'>
支持JSON序列化: {"product_name":"iPhone 15 Pro","reviewer_name":"小王","rating":4,"sentiment":"positive","confidence":...


## 4. TypedDict - 轻量级替代方案

### 什么是 TypedDict？

**TypedDict** 是Python内置的类型注解，比Pydantic更轻量：

**对比**：
- **Pydantic**: 功能强大，有验证、序列化等功能，但较重
- **TypedDict**: 轻量级，只有类型注解，性能更好

### 使用场景

- **简单结构**：只需要类型提示，不需要复杂验证
- **性能敏感**：需要最小开销
- **快速原型**：快速定义结构，后续可升级为Pydantic

In [9]:
# TypedDict 示例
print("【TypedDict 轻量级方案】")

# 定义 TypedDict 结构
class SimplePersonInfo(TypedDict):
    """使用TypedDict定义的简单人员信息"""
    name: str
    age: int
    occupation: str
    skills: List[str]

# 创建结构化模型
simple_model = model.with_structured_output(SimplePersonInfo)

print("TypedDict 模型创建完成")

# 测试
test_info = "李四是一名28岁的数据科学家，擅长机器学习和统计分析"

result = simple_model.invoke(f"分析以下人员信息：{test_info}")

print(f"\n【TypedDict 结果】")
print(f"类型: {type(result)}")
print(f"内容: {result}")
print(f"姓名: {result['name']}")  # 注意：TypedDict使用字典语法
print(f"年龄: {result['age']} (类型: {type(result['age'])})")
print(f"技能: {result['skills']}")

print(f"\n【对比总结】")
print("Pydantic模型:")
print("  - 功能: 丰富（验证、默认值、文档等）")
print("  - 访问: result.field_name")  
print("  - 适用: 生产环境，复杂验证")
print("TypedDict:")
print("  - 功能: 基础（仅类型注解）")
print("  - 访问: result['field_name']")
print("  - 适用: 简单场景，高性能需求")


【TypedDict 轻量级方案】
TypedDict 模型创建完成

【TypedDict 结果】
类型: <class 'dict'>
内容: {'name': '李四', 'age': 28, 'occupation': '数据科学家', 'skills': ['机器学习', '统计分析']}
姓名: 李四
年龄: 28 (类型: <class 'int'>)
技能: ['机器学习', '统计分析']

【对比总结】
Pydantic模型:
  - 功能: 丰富（验证、默认值、文档等）
  - 访问: result.field_name
  - 适用: 生产环境，复杂验证
TypedDict:
  - 功能: 基础（仅类型注解）
  - 访问: result['field_name']
  - 适用: 简单场景，高性能需求


## 5. include_raw 参数 - 获取原始响应

### 什么是 include_raw？

有时我们既需要**结构化数据**，又需要**原始AI响应**：

- **结构化数据**：用于程序处理
- **原始响应**：用于调试、日志记录或展示给用户

`include_raw=True` 让我们同时获得两者。

### 返回格式

```python
# include_raw=False (默认)
result = PersonInfo(name="...", age=25, ...)

# include_raw=True  
result = {
    "parsed": PersonInfo(name="...", age=25, ...),  # 结构化数据
    "raw": AIMessage(content="...", ...)             # 原始响应
}
```

In [10]:
# include_raw 参数演示
print("【include_raw 参数使用】")

# 创建带原始响应的结构化模型
raw_model = model.with_structured_output(PersonInfo, include_raw=True)

test_info = "王五是一名42岁的产品经理，有10年互联网经验，擅长用户研究和产品设计"

print("测试信息：", test_info)
print("\n" + "=" * 50)

# 调用并获得包含原始响应的结果
result = raw_model.invoke(f"分析以下人员信息：{test_info}")

print("【include_raw=True 的结果结构】")
print(f"结果类型: {type(result)}")
print(f"包含的键: {list(result.keys())}")

print(f"\n【结构化数据 (parsed)】")
parsed_data = result["parsed"]
print(f"类型: {type(parsed_data)}")
print(f"姓名: {parsed_data.name}")
print(f"年龄: {parsed_data.age}")
print(f"职业: {parsed_data.occupation}")
print(f"技能: {parsed_data.skills}")

print(f"\n【原始响应 (raw)】")
raw_response = result["raw"]
print(f"类型: {type(raw_response)}")
print(f"响应内容: {raw_response.content}")
print(f"Token使用情况: {getattr(raw_response, 'usage_metadata', '不支持')}")
print(f"响应时间等元数据: {getattr(raw_response, 'response_metadata', '不支持')}")

print(f"\n【实际应用场景】")
print("1. 日志记录：保存原始AI响应用于调试")
print("2. 用户展示：既显示结构化结果，又显示AI的原始解释")  
print("3. 质量监控：分析AI响应的质量和一致性")
print("4. Token统计：监控API使用成本")
print("5. 错误处理：当解析失败时，查看原始响应找原因")

【include_raw 参数使用】
测试信息： 王五是一名42岁的产品经理，有10年互联网经验，擅长用户研究和产品设计

【include_raw=True 的结果结构】
结果类型: <class 'dict'>
包含的键: ['raw', 'parsed', 'parsing_error']

【结构化数据 (parsed)】
类型: <class '__main__.PersonInfo'>
姓名: 王五
年龄: 42
职业: 产品经理
技能: ['用户研究', '产品设计']

【原始响应 (raw)】
类型: <class 'langchain_core.messages.ai.AIMessage'>
响应内容: {"age":42,"name":"王五","occupation":"产品经理","skills":["用户研究","产品设计"]}
Token使用情况: {'input_tokens': 34, 'output_tokens': 24, 'total_tokens': 58, 'input_token_details': {}, 'output_token_details': {}}
响应时间等元数据: {'token_usage': {'completion_tokens': 24, 'prompt_tokens': 34, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None, 'content_tokens': 24}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None, 'text_tokens': 27}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'id': 'chatcmpl-89DmkXn3GEd62qMHX4Zp5hyvvXBI1

## 6. 复杂嵌套结构

### 现实场景的复杂性

实际应用中，数据往往是**多层嵌套**的：

- **公司信息** → 包含多个部门 → 每个部门有多名员工
- **订单信息** → 包含多个商品 → 每个商品有多种属性
- **文档分析** → 包含多个章节 → 每章节有多个要点

Pydantic支持任意复杂的嵌套结构！

### 嵌套模型设计原则

1. **分层定义**：从最内层开始定义模型
2. **组合使用**：外层模型引用内层模型
3. **类型明确**：使用 `List[InnerModel]` 表示列表
4. **描述清晰**：每层都要有准确的 `description`


In [13]:
# 复杂嵌套结构示例
print("【复杂嵌套结构演示】")

# 从内层开始定义模型

# 1. 最内层：员工信息
class Employee(BaseModel):
    """员工信息"""
    name: str = Field(description="员工姓名")
    position: str = Field(description="职位") 
    experience_years: int = Field(description="工作经验年数", ge=0)
    skills: List[str] = Field(description="专业技能")

# 2. 中层：部门信息
class Department(BaseModel):
    """部门信息"""
    name: str = Field(description="部门名称")
    manager: str = Field(description="部门经理姓名")
    employee_count: int = Field(description="员工总数", ge=0)
    employees: List[Employee] = Field(description="部门员工列表")
    budget: Optional[float] = Field(default=None, description="部门预算（万元）", ge=0)

# 3. 外层：公司信息
class CompanyInfo(BaseModel):
    """公司信息"""
    company_name: str = Field(description="公司名称")
    founded_year: int = Field(description="成立年份", ge=1800, le=2024)
    headquarters: str = Field(description="总部地址")
    departments: List[Department] = Field(description="部门列表", min_length=1)
    total_employees: int = Field(description="员工总数", ge=0)
    
    # 自定义验证器
    @validator('total_employees')
    def validate_total_employees(cls, v, values):
        """验证员工总数与各部门员工数的一致性"""
        if 'departments' in values:
            dept_total = sum(dept.employee_count for dept in values['departments'])
            if v != dept_total:
                print(f"警告：总员工数({v})与各部门员工数总和({dept_total})不一致")
        return v

# 创建复杂嵌套结构的分析器
company_analyzer = model.with_structured_output(CompanyInfo)

# 测试复杂的公司信息
complex_company_text = """
科技创新有限公司成立于2018年，总部位于深圳南山区。公司目前有85名员工，分为三个部门：

技术部门由张三管理，有35名员工，包括：
- 李四：高级工程师，5年经验，擅长Python、机器学习、云计算
- 王五：前端工程师，3年经验，擅长React、Vue、TypeScript
- 赵六：数据科学家，4年经验，擅长数据分析、AI算法、统计建模

产品部门由陈七管理，有25名员工，包括：  
- 孙八：产品经理，6年经验，擅长需求分析、用户体验、项目管理
- 周九：UI设计师，4年经验，擅长界面设计、用户研究、原型设计

销售部门由吴十管理，有25名员工，包括：
- 郑一：销售总监，8年经验，擅长客户管理、商务谈判、市场开拓
- 刘二：市场专员，2年经验，擅长市场调研、品牌推广、活动策划
"""

print("分析复杂的公司信息...")
print("文本长度:", len(complex_company_text), "字符")
print("\n" + "=" * 60)

# 分析复杂结构
result = company_analyzer.invoke(f"请详细分析这个公司信息，提取所有员工和部门数据：\n{complex_company_text}")

print("\n【复杂嵌套结构分析结果】")
print(f"公司名称: {result.company_name}")
print(f"成立年份: {result.founded_year}")  
print(f"总部地址: {result.headquarters}")
print(f"员工总数: {result.total_employees}")
print(f"部门数量: {len(result.departments)}")

print(f"\n【各部门详细信息】")
for i, dept in enumerate(result.departments, 1):
    print(f"\n{i}. {dept.name}")
    print(f"   经理: {dept.manager}")
    print(f"   员工数: {dept.employee_count}")
    print(f"   预算: {dept.budget or '未提及'}万元")
    
    for j, emp in enumerate(dept.employees, 1):
        print(f"   员工{j}: {emp.name}")
        print(f"     职位: {emp.position}")
        print(f"     经验: {emp.experience_years}年")
        print(f"     技能: {', '.join(emp.skills)}")

print(f"\n【数据验证检查】")
calculated_total = sum(dept.employee_count for dept in result.departments)
print(f"各部门员工数总和: {calculated_total}")
print(f"声称的员工总数: {result.total_employees}")
print(f"数据一致性: {'✅ 一致' if calculated_total == result.total_employees else '❌ 不一致'}")

print(f"\n结构化对象类型: {type(result)}")
print(f"JSON序列化大小: {len(result.model_dump_json())} 字符")

【复杂嵌套结构演示】
分析复杂的公司信息...
文本长度: 350 字符



/tmp/ipykernel_27823/3235258980.py:33: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('total_employees')



【复杂嵌套结构分析结果】
公司名称: 科技创新有限公司
成立年份: 2018
总部地址: 深圳南山区
员工总数: 85
部门数量: 3

【各部门详细信息】

1. 技术部门
   经理: 张三
   员工数: 35
   预算: 未提及万元
   员工1: 李四
     职位: 高级工程师
     经验: 5年
     技能: Python, 机器学习, 云计算
   员工2: 王五
     职位: 前端工程师
     经验: 3年
     技能: React, Vue, TypeScript
   员工3: 赵六
     职位: 数据科学家
     经验: 4年
     技能: 数据分析, AI算法, 统计建模

2. 产品部门
   经理: 陈七
   员工数: 25
   预算: 未提及万元
   员工1: 孙八
     职位: 产品经理
     经验: 6年
     技能: 需求分析, 用户体验, 项目管理
   员工2: 周九
     职位: UI设计师
     经验: 4年
     技能: 界面设计, 用户研究, 原型设计

3. 销售部门
   经理: 吴十
   员工数: 25
   预算: 未提及万元
   员工1: 郑一
     职位: 销售总监
     经验: 8年
     技能: 客户管理, 商务谈判, 市场开拓
   员工2: 刘二
     职位: 市场专员
     经验: 2年
     技能: 市场调研, 品牌推广, 活动策划

【数据验证检查】
各部门员工数总和: 85
声称的员工总数: 85
数据一致性: ✅ 一致

结构化对象类型: <class '__main__.CompanyInfo'>
JSON序列化大小: 950 字符


## 7. 与 LCEL 结合使用

### 强大的组合

结构化输出 + LCEL = **极其强大的数据处理管道**！

**组合优势**：
- **类型安全的Pipeline**：每个环节都有明确的输入输出类型
- **可组合的处理链**：轻松构建复杂的数据处理流程
- **自动验证**：数据在管道中流动时自动验证
- **易于调试**：每个环节都有结构化的中间结果

### 典型应用模式

```python
# 模式1: 多步骤处理
input_processor | structured_model | output_formatter

# 模式2: 条件分支处理  
input_validator | structured_analyzer | conditional_processor

# 模式3: 并行处理合并
parallel_processors | result_merger | final_formatter
```


## 8. 实战项目：智能数据提取器

### 项目概述

构建一个**智能数据提取器**，能够从非结构化文本中提取结构化信息。

**功能特点**：
- 支持多种文档类型（简历、新闻、产品介绍等）
- 自动识别文档类型
- 提取对应的结构化数据
- 生成标准化的JSON输出
- 支持批量处理和错误处理

**技术栈**：
- 结构化输出（多种Schema）
- LCEL 组合（数据处理Pipeline）
- 错误处理和验证


In [14]:
# 实战项目：智能数据提取器
print("【智能数据提取器实战项目】")

from langchain_core.runnables import chain as chain_decorator
from enum import Enum

# 1. 定义文档类型枚举
class DocumentType(str, Enum):
    RESUME = "resume"
    NEWS = "news" 
    PRODUCT = "product"
    UNKNOWN = "unknown"

# 2. 文档类型识别模型
class DocumentClassification(BaseModel):
    """文档类型分类结果"""
    document_type: DocumentType = Field(description="文档类型")
    confidence: float = Field(description="分类置信度", ge=0.0, le=1.0)
    reasoning: str = Field(description="分类依据")

# 3. 不同文档类型的Schema

# 简历Schema
class ResumeData(BaseModel):
    """简历信息结构"""
    name: str = Field(description="姓名")
    phone: Optional[str] = Field(default=None, description="电话号码")
    email: Optional[str] = Field(default=None, description="邮箱地址")
    education: List[str] = Field(description="教育背景")
    work_experience: List[str] = Field(description="工作经历")
    skills: List[str] = Field(description="技能列表")
    years_experience: Optional[int] = Field(default=None, description="总工作年限", ge=0)

# 新闻Schema  
class NewsData(BaseModel):
    """新闻信息结构"""
    title: str = Field(description="新闻标题")
    summary: str = Field(description="新闻摘要")
    key_people: List[str] = Field(description="涉及的关键人物")
    companies: List[str] = Field(description="涉及的公司/机构")
    locations: List[str] = Field(description="涉及的地点")
    date_mentioned: Optional[str] = Field(default=None, description="提到的日期")
    category: str = Field(description="新闻类别")

# 产品Schema
class ProductData(BaseModel):
    """产品信息结构"""
    product_name: str = Field(description="产品名称")
    brand: str = Field(description="品牌")
    category: str = Field(description="产品类别") 
    price: Optional[str] = Field(default=None, description="价格信息")
    key_features: List[str] = Field(description="主要特性")
    target_audience: str = Field(description="目标用户群体")
    availability: Optional[str] = Field(default=None, description="上市时间/可用性")

# 4. 创建分类器
classifier = model.with_structured_output(DocumentClassification)

# 5. 创建不同类型的数据提取器  
resume_extractor = model.with_structured_output(ResumeData)
news_extractor = model.with_structured_output(NewsData)
product_extractor = model.with_structured_output(ProductData)

# 6. 使用LCEL构建智能提取Pipeline

@chain_decorator
def document_preprocessor(text_input):
    """文档预处理"""
    text = text_input.get("text", "").strip()
    if not text:
        raise ValueError("输入文本不能为空")
    
    return {
        "original_text": text,
        "cleaned_text": text.replace("\n\n", "\n").strip(),
        "char_count": len(text),
        "estimated_processing_time": len(text) // 1000  # 每1000字符估算1秒
    }

@chain_decorator  
def classify_document(processed_data):
    """文档分类"""
    text = processed_data["cleaned_text"]
    
    classification_result = classifier.invoke(
        f"请分析以下文档的类型（简历/新闻/产品介绍）：\n\n{text}"
    )
    
    return {
        **processed_data,
        "classification": classification_result
    }

@chain_decorator
def extract_structured_data(classified_data):
    """根据文档类型提取结构化数据"""
    text = classified_data["cleaned_text"]
    doc_type = classified_data["classification"].document_type
    
    extractors = {
        DocumentType.RESUME: resume_extractor,
        DocumentType.NEWS: news_extractor, 
        DocumentType.PRODUCT: product_extractor
    }
    
    if doc_type in extractors:
        extractor = extractors[doc_type]
        extracted_data = extractor.invoke(
            f"请从以下{doc_type}文档中提取结构化信息：\n\n{text}"
        )
    else:
        extracted_data = {"error": f"未支持的文档类型: {doc_type}"}
    
    return {
        "document_type": doc_type,
        "classification_confidence": classified_data["classification"].confidence,
        "extracted_data": extracted_data,
        "processing_stats": {
            "char_count": classified_data["char_count"],
            "estimated_time": classified_data["estimated_processing_time"]
        }
    }

# 7. 组合完整的Pipeline
smart_extractor_pipeline = (
    document_preprocessor 
    | classify_document 
    | extract_structured_data
)

print("智能数据提取器Pipeline构建完成！")
print("支持的文档类型:", [dt.value for dt in DocumentType])

# 8. 测试不同类型的文档

# 测试简历
resume_text = """
张三的个人简历

基本信息：
姓名：张三
电话：138-0013-8000  
邮箱：zhangsan@example.com

教育背景：
2018-2022 清华大学计算机科学与技术学士
2022-2024 北京大学软件工程硕士

工作经历：
2024至今 - 阿里巴巴 - 高级软件工程师
主要负责推荐系统的算法优化，使用Python和机器学习技术

2022-2024 - 腾讯 - 软件工程师（实习）
参与微信小程序后端开发，掌握Java、Spring框架

技能：
Python、Java、机器学习、数据分析、Spring、MySQL、Redis
"""

# 测试新闻
news_text = """
马斯克宣布特斯拉上海工厂产能提升50%

【上海，2024年1月15日】特斯拉CEO埃隆·马斯克今日在上海超级工厂宣布，该工厂的年产能将从目前的75万辆提升至112万辆，增幅达50%。

马斯克表示，此次产能提升主要得益于生产线优化和新技术应用。特斯拉上海工厂是该公司在中国的重要生产基地，主要生产Model 3和Model Y车型。

上海市政府代表出席了此次发布会，并表示将继续支持特斯拉在华发展。预计产能提升将在2024年第三季度完成，届时将为中国市场提供更充足的车源。

这一消息发布后，特斯拉股价在纳斯达克交易所上涨3.2%。
"""

# 测试产品介绍  
product_text = """
iPhone 15 Pro Max - 重新定义专业级智能手机

苹果公司最新发布的iPhone 15 Pro Max搭载了全新的A17 Pro芯片，采用3纳米工艺制程，性能较上一代提升20%。

主要特性：
• 6.7英寸超视网膜XDR显示屏，支持120Hz自适应刷新率
• 4800万像素主摄像头，支持5倍光学变焦
• 钛金属材质机身，重量减轻19克
• 支持USB-C接口，数据传输速度提升10倍
• 最长29小时视频播放续航

价格：256GB版本9999元起，512GB版本11999元，1TB版本13999元

目标用户：专业摄影师、内容创作者、商务人士以及科技爱好者

预计将于9月22日正式发售，现已开放预订。
"""

# 批量测试
test_documents = [
    {"name": "简历", "text": resume_text},
    {"name": "新闻", "text": news_text}, 
    {"name": "产品介绍", "text": product_text}
]

print("\n" + "=" * 60)
print("开始批量测试智能数据提取...")

for i, doc in enumerate(test_documents, 1):
    print(f"\n【测试文档 {i}: {doc['name']}】")
    
    try:
        result = smart_extractor_pipeline.invoke({"text": doc["text"]})
        
        print(f"文档类型: {result['document_type']}")
        print(f"分类置信度: {result['classification_confidence']:.2f}")
        print(f"字符数: {result['processing_stats']['char_count']}")
        
        print("\n提取的结构化数据:")
        extracted = result['extracted_data']
        if isinstance(extracted, dict) and 'error' in extracted:
            print(f"  ❌ {extracted['error']}")
        else:
            for key, value in extracted.model_dump().items():
                print(f"  {key}: {value}")
                
    except Exception as e:
        print(f"❌ 处理失败: {e}")
    
    print("-" * 50)

print("\n✅ 智能数据提取器测试完成！")


【智能数据提取器实战项目】
智能数据提取器Pipeline构建完成！
支持的文档类型: ['resume', 'news', 'product', 'unknown']

开始批量测试智能数据提取...

【测试文档 1: 简历】
文档类型: resume
分类置信度: 0.95
字符数: 279

提取的结构化数据:
  name: 张三
  phone: 138-0013-8000
  email: zhangsan@example.com
  education: ['2018-2022 清华大学计算机科学与技术学士', '2022-2024 北京大学软件工程硕士']
  work_experience: ['2024至今 - 阿里巴巴 - 高级软件工程师 - 主要负责推荐系统的算法优化，使用Python和机器学习技术', '2022-2024 - 腾讯 - 软件工程师（实习） - 参与微信小程序后端开发，掌握Java、Spring框架']
  skills: ['Python', 'Java', '机器学习', '数据分析', 'Spring', 'MySQL', 'Redis']
  years_experience: 2
--------------------------------------------------

【测试文档 2: 新闻】
文档类型: news
分类置信度: 0.95
字符数: 268

提取的结构化数据:
  title: 马斯克宣布特斯拉上海工厂产能提升50%
  summary: 特斯拉CEO埃隆·马斯克宣布上海工厂年产能将提升50%，从75万辆增至112万辆，预计2024年第三季度完成。
  key_people: ['埃隆·马斯克']
  companies: ['特斯拉']
  locations: ['上海']
  date_mentioned: 2024年1月15日
  category: 汽车
--------------------------------------------------

【测试文档 3: 产品介绍】
文档类型: product
分类置信度: 0.95
字符数: 313

提取的结构化数据:
  product_name: iPhone 15 Pro Max
  brand: 苹果公司


## 本章总结

### 核心知识点回顾

**结构化输出的重要性**
- 解决AI输出格式不一致的问题
- 提供类型安全和自动验证
- 让AI输出变得可编程和可靠

**with_structured_output() 核心方法**
- LangChain v1的标准结构化输出方式
- 支持 Pydantic 模型和 TypedDict
- 可配置 include_raw 获取原始响应

**Pydantic 模型功能**
- 强大的类型注解和验证
- Field() 提供丰富的约束条件
- 支持复杂嵌套结构
- 自动生成JSON Schema

**TypedDict 轻量替代**
- 适用于简单场景
- 更好的性能表现
- 基础的类型提示功能

**与 LCEL 的强大结合**
- 构建类型安全的处理管道
- 支持复杂的数据流转换
- 易于组合和扩展

### 最佳实践

**1. 模型设计**
- 字段描述要清晰具体
- 合理使用 Optional 和默认值
- 添加适当的验证约束
- 考虑字段的实际业务含义

**2. 性能优化**
- 简单场景使用 TypedDict
- 复杂验证使用 Pydantic
- 合理设置模型 temperature（推荐0.1-0.3）
- 考虑使用 include_raw=False 减少开销

**3. 错误处理**
- 总是捕获解析异常
- 使用 include_raw 调试问题
- 验证关键字段的存在
- 设计合理的降级策略

**4. 与传统方式的对比**

| 方面 | 传统文本解析 | 结构化输出 |
|------|-------------|-----------|
| **可靠性** | ❌ 格式易变 | ✅ 格式固定 |
| **类型安全** | ❌ 手动转换 | ✅ 自动验证 |
| **开发效率** | ❌ 复杂解析 | ✅ 直接使用 |
| **维护成本** | ❌ 频繁修复 | ✅ 稳定可靠 |
| **IDE支持** | ❌ 无提示 | ✅ 完整支持 |

### 学习成果检验

通过本章学习，你应该能够：
- ✅ 解释结构化输出的优势和应用场景
- ✅ 使用 Pydantic 定义复杂的数据模型
- ✅ 掌握 with_structured_output() 的各种用法
- ✅ 构建类型安全的 LCEL 处理管道
- ✅ 处理复杂嵌套结构的数据提取
- ✅ 在实际项目中应用结构化输出技术

### 与前几章的关系

```
第01章：Chat Models基础
  ↓ 掌握模型调用
第02章：Prompts & Messages  
  ↓ 掌握提示词设计
第03章：LCEL基础
  ↓ 掌握组件组合
第04章：Structured Output
  ↓ 掌握结构化输出（当前）
第05章：Tools & Tool Calling
```